In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
learning_rate = 1e-4
epochs=20
batch_size = 32

In [3]:
directory = r"/Users/rituraj/Desktop/Internship/RealOrFake Face/dataset"
categories = ["fake","real"]

In [4]:
data=[]
labels=[]
for category in categories:
    path = os.path.join(directory, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        image = load_img(img_path, target_size=(224, 224))
        image = img_to_array(image)
        image = preprocess_input(image)

        data.append(image)
        labels.append(category)

In [5]:
#ONE HOT ENCODING ON LABELS
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [6]:
data = np.array(data,dtype='float32')
labels = np.array(labels)

In [7]:
#SPLITTING
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.20,stratify=labels , random_state=42)

In [8]:
#IMAGE DATA CLASS FOR IMAGE AUGMENTATION
augm = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

In [9]:
#MOBILENETS MODEL
baseM = MobileNetV2(weights="imagenet", include_top=False,
    input_tensor=Input(shape=(224, 224, 3)))

Metal device set to: Apple M1


2022-01-21 01:09:12.603067: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-01-21 01:09:12.603977: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [10]:
#MODEL Head
finalM = baseM.output
finalM = AveragePooling2D(pool_size=(7, 7))(finalM)
finalM = Flatten(name="flatten")(finalM)
finalM = Dense(128, activation="relu")(finalM)
finalM = Dropout(0.5)(finalM)
finalM = Dense(2, activation="softmax")(finalM)

In [11]:
#FINAL MODEL
model = Model(inputs=baseM.input,outputs=finalM)

In [12]:
for layer in baseM.layers:
    layer.trainable = False

In [13]:
#Compiling Model
opt = Adam(learning_rate=learning_rate, decay=learning_rate / epochs)
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])

In [ ]:
output = model.fit(X_train,y_train,batch_size=batch_size,
                  steps_per_epoch=len(X_train)//batch_size,
                  validation_data=(X_train,y_train),
                  validation_steps=len(X_test)//batch_size,
                  epochs = epochs)

2022-01-21 01:09:14.477329: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/20


2022-01-21 01:09:15.791528: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


51/51 [==============================] - ETA: 0s - loss: 0.7727 - accuracy: 0.5214

2022-01-21 01:09:30.917594: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


51/51 [==============================] - 20s 342ms/step - loss: 0.7727 - accuracy: 0.5214 - val_loss: 0.6843 - val_accuracy: 0.5703
Epoch 2/20
51/51 [==============================] - 16s 321ms/step - loss: 0.7116 - accuracy: 0.5656 - val_loss: 0.6637 - val_accuracy: 0.5990
Epoch 3/20
51/51 [==============================] - 16s 324ms/step - loss: 0.6874 - accuracy: 0.5962 - val_loss: 0.6475 - val_accuracy: 0.6458
Epoch 4/20
51/51 [==============================] - 17s 332ms/step - loss: 0.6628 - accuracy: 0.6183 - val_loss: 0.6342 - val_accuracy: 0.6458
Epoch 5/20
51/51 [==============================] - 17s 344ms/step - loss: 0.6500 - accuracy: 0.6311 - val_loss: 0.6270 - val_accuracy: 0.6797
Epoch 6/20
51/51 [==============================] - 17s 341ms/step - loss: 0.6466 - accuracy: 0.6366 - val_loss: 0.6174 - val_accuracy: 0.7031
Epoch 7/20
51/51 [==============================] - 17s 339ms/step - loss: 0.6281 - accuracy: 0.6538 - val_loss: 0.6026 - val_accuracy: 0.6693
Epoch 8/20

In [ ]:
predict = model.predict(X_test,batch_size = batch_size)

In [ ]:
predict_l = np.argmax(predict,axis=1)
print(classification_report(y_test.argmax(axis=1),predict_l,target_names = lb.classes_))

In [ ]:
N = epochs
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), output.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), output.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), output.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), output.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot.png")

In [ ]:
#Single Prediction
image = load_img('testing/t4r.jpg', target_size=(224, 224))
image = img_to_array(image)
image = preprocess_input(image)
sdf = []
sdf.append(image)
sdf = np.array(sdf,dtype='float32')

single_prediction = model.predict(sdf)
predict_t = np.argmax(single_prediction,axis=1)
answer = lb.inverse_transform(predict_t)

if answer=='fake':
    print('Fake')
else:
    print('Real')

In [ ]:
model.save("RealOrFakeFaceModel.model", save_format="h5")